In [1]:
from glob import glob
from tqdm.auto import tqdm
import polars

embeddings = glob('../data/q?_embeddings.parquet')
tables = [polars.read_parquet(x) for x in embeddings]
df = polars.concat(tables)
df = df.select('vector_id')
unique_ids = df['vector_id'].unique().sort().to_list()

In [2]:
import chromadb
arg = chromadb.PersistentClient('./persist').get_collection('imf_publications_arg')

In [3]:
vectors = arg.get(ids=unique_ids)

In [4]:
polars.DataFrame([
    vectors['metadatas'][i] 
    | dict(
        vector_id=vectors['ids'][i],
        text=vectors['documents'][i]
    )
    
    for i in 
    range(len(vectors['metadatas']))
]).write_parquet('../data/arg_embeddings.parquet')